In [5]:
import logging
logging.basicConfig(level=logging.ERROR)
import nest_asyncio
nest_asyncio.apply()

import numpy as np

from page_reader import ConcurrentWrapper, SimplePageReader
from rerankers import Reranker
from rewriters import QwenRewriter
from search_engines.bing import BingRequest, BingTextSearch
from splitters import split_search_records

def format_information(search_records):
    knowledge_template = """
    <Search Record>
    Title: {title}
    URL: {url}
    Content: {content}
    </Search Record>
    """.strip()

    return "\n\n".join([knowledge_template.format(
        title = s.title.strip(),
        url = s.url.strip(), 
        content = s.content.strip()) 
    for s in search_records])


def execute_search_engine(q:str, topn:int = 5):
    # 构建请求
    text_req = BingRequest.chinese(q)
    
    # 执行搜索
    rewriter = QwenRewriter()
    text_searcher = BingTextSearch(rewriter = rewriter)
    search_records = text_searcher.search_normalize(text_req)
    
    # 详情爬虫
    page_reader = ConcurrentWrapper(reader = SimplePageReader())
    urls = [r.url for r in search_records]
    contents = page_reader.read(urls)
    for i, s in enumerate(search_records):
        s.set_content(contents[i])
       
    # 内容分割 
    chunked_records = split_search_records(search_records)
    
    # 内容精排
    reranker = Reranker()
    
    docs = [s.content for s in chunked_records]
    scores = reranker.rerank(q, docs)
    top_records = np.array(chunked_records)[np.argsort(scores)[::-1]][:topn].tolist()
        
    return top_records


def search_tool(query:str, topn:int = 5) -> str:
    """Search microsoft bing against a query, rerank results and return topn results as a knowledge string."""
    search_records = execute_search_engine(query, topn)
    return format_information(search_records)

In [9]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama

bing_tool = FunctionTool.from_defaults(
    search_tool
)

agent = ReActAgent.from_tools(
    tools = [bing_tool], 
    llm=Ollama(model="qwen2.5:14b"),
    verbose=True,
    context = "Today is 2024-11-26"
)

In [11]:
agent.chat(message = "你知道大学生摇滚联盟么？")

> Running step 7e85164f-b5fb-49c3-a788-75939c61282c. Step input: 你知道大学生摇滚联盟么？
Thought: 需要查询一下“大学生摇滚联盟”的相关信息。
Action: search_tool
Action Input: {'query': '大学生摇滚联盟'}
Using query:  大学生摇滚乐队联盟活动
Reranking 86 documents...
Reranking taking 0.53 seconds
Observation: <Search Record>
    Title: 中国大学生摇滚联盟_百度百科
    URL: https://baike.baidu.com/item/%E4%B8%AD%E5%9B%BD%E5%A4%A7%E5%AD%A6%E7%94%9F%E6%91%87%E6%BB%9A%E8%81%94%E7%9B%9F/4746430
    Content: 2004年5月我组建了“大学生摇滚联盟”。作为一个公益组织，我们的出发点是给各个大学的校园乐队提供各种便利，包括演出机会、录音等等，并从中发掘优秀的乐队协助他们发展，为中国大学的摇滚乐创造更好的氛围，该组织主要在昌平地区活动，旗下有6支签约乐队，他们分别来自中国石油大学、中国政法大学、北京汇佳职业学院、北京信息科技学院等学校，他们正在大学生摇滚联盟的协助下积极的进行创作、排练与演出。2009年10月，为了更好的发展，建立了大学生摇滚联盟论坛，并进行了精心的维护与发展，取得了不错的反响。
    </Search Record>

<Search Record>
    Title: 【学生会】小蓝鲸音乐节|永远年轻，永远热泪盈眶，让星空和 ...
    URL: https://tuanwei.nju.edu.cn/36/71/c26265a407153/page.htm
    Content: 9月17日晚7点，由南京大学学生会主办、南京大学摇滚联盟协办的首届“小蓝鲸音乐节”在仙林校区炜华运动场准点开启。
近千名南大学生来到体育场参与活动，校团委书记葛俊杰老师、校团委副书记周媛老师等许多老师也亲临现场，感受青春的脉动与 ...
    </Search Record>

<Search Rec

AgentChatResponse(response='大学生摇滚联盟是由我方在2004年5月创建的一个公益组织，其目标是给各个大学的校园乐队提供各种便利和服务，并协助他们发展和推广。它汇聚了来自中国石油大学、中国政法大学等多所学校的6支签约乐队，并且在昌平地区活动。此外，在2019年成立了全国大学生音乐社团联盟，以北京和南京的学校为核心，汇聚了全国超过一百所大学的音乐社团。', sources=[ToolOutput(content='<Search Record>\n    Title: 中国大学生摇滚联盟_百度百科\n    URL: https://baike.baidu.com/item/%E4%B8%AD%E5%9B%BD%E5%A4%A7%E5%AD%A6%E7%94%9F%E6%91%87%E6%BB%9A%E8%81%94%E7%9B%9F/4746430\n    Content: 2004年5月我组建了“大学生摇滚联盟”。作为一个公益组织，我们的出发点是给各个大学的校园乐队提供各种便利，包括演出机会、录音等等，并从中发掘优秀的乐队协助他们发展，为中国大学的摇滚乐创造更好的氛围，该组织主要在昌平地区活动，旗下有6支签约乐队，他们分别来自中国石油大学、中国政法大学、北京汇佳职业学院、北京信息科技学院等学校，他们正在大学生摇滚联盟的协助下积极的进行创作、排练与演出。2009年10月，为了更好的发展，建立了大学生摇滚联盟论坛，并进行了精心的维护与发展，取得了不错的反响。\n    </Search Record>\n\n<Search Record>\n    Title: 【学生会】小蓝鲸音乐节|永远年轻，永远热泪盈眶，让星空和 ...\n    URL: https://tuanwei.nju.edu.cn/36/71/c26265a407153/page.htm\n    Content: 9月17日晚7点，由南京大学学生会主办、南京大学摇滚联盟协办的首届“小蓝鲸音乐节”在仙林校区炜华运动场准点开启。\n近千名南大学生来到体育场参与活动，校团委书记葛俊杰老师、校团委副书记周媛老师等许多老师也亲临现场，感受青春的脉动与 ...\n    </Search Record>\n\n<Search Record>\n    Title: 本周末，28所高